In [1]:
#Load libraries
from __future__ import print_function
from __future__ import division
import numpy as np 
import matplotlib
import matplotlib.pyplot as plt 
from tqdm import trange, tqdm
import pickle

from hackathon.utils.utils import *

%matplotlib inline
matplotlib.rcParams['figure.figsize'] = (5, 5)

%load_ext autoreload
%autoreload 2

In [2]:
from sklearn.tree import ExtraTreeRegressor
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
from sklearn.ensemble import AdaBoostRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.linear_model import ElasticNet, Ridge
from sklearn.linear_model import HuberRegressor, RANSACRegressor, SGDRegressor
from sklearn.multioutput import MultiOutputRegressor

from sklearn.metrics import mean_squared_error

import xgboost as xgb


from keras.models import *
from keras.layers import *
from keras.regularizers import *
from keras.optimizers import *
from keras.metrics import *
from keras.activations import *
from keras.losses import *
from keras.objectives import *
from keras.wrappers.scikit_learn import KerasRegressor

/home/kinshiryuu-burp/anaconda3/envs/nitinol/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
Using TensorFlow backend.


In [3]:
from hackathon.utils.utils import *
from hackathon.utils.draw_utils import *
import pandas as pd

In [4]:
#Load prepared data
test_data = parse_data("../data/test_data.csv")
train_data = parse_data("../data/train_data.csv")

In [5]:
train_data.shape

(55375, 81)

In [6]:
train_data.tail(500)

,NPWD2372,NPWD2401,NPWD2402,NPWD2451,NPWD2471,NPWD2472,NPWD2481,NPWD2482,NPWD2491,NPWD2501,...,D3POCM,D2PLND,D5PPHB,D7PLTS,D8PLTP,SPOT,D9PSPO,usbx,usby,usbz
ut_ms,,,,,,,,,,,,,,,,,,,,,
2014-12-11 03:00:00,0.153997,0.001548,0.175747,0.413997,0.000741,0.002018,0.000573,0.001915,0.250222,0.005270,...,0,0,0,0,0,0,0,-0.885074,-0.015705,-0.465185
2014-12-11 04:00:00,0.182451,0.001518,0.175296,0.718407,0.000736,0.001916,0.000567,0.001911,0.293467,0.005268,...,0,0,0,0,0,0,0,0.229725,-0.297169,0.926778
2014-12-11 05:00:00,0.068337,0.001524,0.175418,0.605100,0.000745,0.001784,0.000577,0.001912,0.109192,0.005261,...,0,0,0,0,0,0,0,0.682524,0.585007,-0.438096
2014-12-11 06:00:00,0.067819,0.001518,0.175529,0.916710,0.000745,0.001869,0.000577,0.001911,0.153566,0.005179,...,0,0,0,0,0,0,0,0.693310,0.493665,-0.524992
2014-12-11 07:00:00,0.091699,0.001513,0.175465,0.238846,0.000739,0.001739,0.000571,0.001915,0.154937,0.005243,...,0,0,0,0,0,0,0,-0.343933,0.397570,-0.850675
2014-12-11 08:00:00,0.103793,0.001529,0.175529,0.509307,0.000745,0.001869,0.000573,0.001901,0.176760,0.005250,...,0,0,0,0,0,0,0,0.757616,0.154589,0.634129
2014-12-11 09:00:00,0.187734,0.001527,0.175230,0.345856,0.000760,0.001736,0.000572,0.001889,0.300233,0.005261,...,0,0,0,0,0,0,0,-0.224307,-0.950424,0.215360
2014-12-11 10:00:00,0.139870,0.001534,0.175762,0.763595,0.000743,0.001821,0.000572,0.001901,0.191653,0.005259,...,0,0,0,0,0,0,0,-0.530428,0.724253,-0.440571
2014-12-11 11:00:00,0.180813,0.001513,0.175089,1.321200,0.000749,0.001545,0.000578,0.001951,0.315071,0.005216,...,0,0,0,0,0,0,0,0.520810,0.113966,-0.846031


In [7]:
train_dates = train_data.index < '2014-12-01 00:00:00'

In [8]:
train_df = train_data[train_dates]
valid_df = train_data[~train_dates]

train_df.shape, valid_df.shape

((54632, 81), (743, 81))

In [9]:
x_columns = filter(lambda x: "NPWD" not in x 
                   and 'ut_ms' not in x, train_df.columns)
x_columns

['sunmars_km',
 'earthmars_km',
 'sunmarsearthangle_deg',
 'solarconstantmars',
 'eclipseduration_min',
 'occultationduration_min',
 'sa',
 'sx',
 'sy',
 'sz',
 'dmop_count_24h_AAAA',
 'dmop_count_24h_AACF',
 'dmop_count_24h_ADMC',
 'dmop_count_24h_AHHH',
 'dmop_count_24h_AMMM',
 'dmop_count_24h_AOOO',
 'dmop_count_24h_APSF',
 'dmop_count_24h_APWF',
 'dmop_count_24h_ASEQ',
 'dmop_count_24h_ASSS',
 'dmop_count_24h_ASXX',
 'dmop_count_24h_ATMB',
 'dmop_count_24h_ATTT',
 'dmop_count_24h_AVVV',
 'dmop_count_24h_AXXX',
 'dmop_count_24h_sum',
 'flagcomms',
 'EARTH',
 'SLEW',
 'NADIR',
 'MAINTENANCE',
 'INERTIAL',
 'ACROSS_TRACK',
 'WARMUP',
 'D1PVMC',
 'RADIO_SCIENCE',
 'SPECULAR',
 'D4PNPO',
 'D3POCM',
 'D2PLND',
 'D5PPHB',
 'D7PLTS',
 'D8PLTP',
 'SPOT',
 'D9PSPO',
 'usbx',
 'usby',
 'usbz']

In [10]:
y_columns = filter(lambda x: "NPWD"  in x, train_df.columns)
y_columns

['NPWD2372',
 'NPWD2401',
 'NPWD2402',
 'NPWD2451',
 'NPWD2471',
 'NPWD2472',
 'NPWD2481',
 'NPWD2482',
 'NPWD2491',
 'NPWD2501',
 'NPWD2531',
 'NPWD2532',
 'NPWD2551',
 'NPWD2552',
 'NPWD2561',
 'NPWD2562',
 'NPWD2691',
 'NPWD2692',
 'NPWD2721',
 'NPWD2722',
 'NPWD2742',
 'NPWD2771',
 'NPWD2791',
 'NPWD2792',
 'NPWD2801',
 'NPWD2802',
 'NPWD2821',
 'NPWD2851',
 'NPWD2852',
 'NPWD2871',
 'NPWD2872',
 'NPWD2881',
 'NPWD2882']

In [11]:
# create train/valid datasets
X_train = train_df[x_columns]
y_train = train_df[y_columns]

X_valid = valid_df[x_columns]
y_valid = valid_df[y_columns]

X_test = test_data[x_columns]

In [12]:
parameter_dict = {    # Parameters for XGBRegressor
    "n_estimators":200,
    "silent":1,
    "nthread":-1,
    "gamma":0.001,
    "max_depth":6,
    "min_child_weight":2,
    "max_delta_step":1,
    "reg_lambda":0.1,
    "reg_alpha":0.1,
    'max_depth': 3,
    'subsample': 0.7,
    'colsample_bylevel': 0.4,
    'colsample_bytree': 0.4,
}

#Create lists to check regressors

names = [
    "ExtraTreesRegressor",
    "RandomForestRegressor",
    #"SVR_linear",
    #"SVR_rbf",
    "ElasticNet",
    "Ridge",
    "HuberRegressor",
    "RANSACRegressor",
    #"SGDRegressor",
    "XGBRegressor",
    "GradientBoostingRegressor"
]

ensemble = [
    "AdaBoostRegressor"
]

regressors = [
    ExtraTreesRegressor(n_estimators=20, criterion='mse', max_depth=10,
                       min_samples_split=3, min_samples_leaf=4, bootstrap=True),
    RandomForestRegressor(n_estimators=20, criterion='mse', max_depth=10,
                         min_samples_split=3, min_samples_leaf=4, bootstrap=True),
    #SVR(kernel='linear', C=10.0, cache_size=400, max_iter=10000),
    #SVR(C=100.0, max_iter=10000),
    ElasticNet(alpha=1.5, l1_ratio=0.1, normalize=True, warm_start=True),
    Ridge(alpha=0.8, normalize=True),
    HuberRegressor(warm_start=True),
    RANSACRegressor(max_trials=150, loss='squared_loss'),
    #SGDRegressor(loss='huber', penalty='elasticnet', alpha=0.001, n_iter=30,
    #            learning_rate='optimal'),
    xgb.XGBRegressor(**parameter_dict),
    GradientBoostingRegressor()
]

In [13]:
X_train = X_train.as_matrix()
y_train = y_train.as_matrix()

X_valid = X_valid.as_matrix()
y_valid = y_valid.as_matrix()

X_test = X_test.as_matrix()

In [14]:
#Own RMSE function - we don't operate at pandas DataFrames
def RMSE_func(y_true, y_pred, axis=None):
    diff = y_true - y_pred
    rmse = np.sqrt(np.mean(diff**2, axis=axis))
    return rmse

In [13]:
losses = {}

for name, reg in zip(names, regressors):
    print("Fitting",name)
    #print("\tUsing",ensemble[0])
    
    #rega = AdaBoostRegressor(reg, n_estimators=70)
    
    regress = MultiOutputRegressor(reg, n_jobs=-1)
    regress.fit(X_train, y_train)
    
    predicts = regress.predict(X_valid)
    losses[name] = RMSE_func(y_valid, predicts)
    
for name in names:
    print("Loss of",name,"=",losses[name])
        

Fitting ExtraTreesRegressor
Fitting RandomForestRegressor
Fitting ElasticNet
Fitting Ridge
Fitting HuberRegressor
Fitting RANSACRegressor
Fitting XGBRegressor
Fitting GradientBoostingRegressor
Loss of ExtraTreesRegressor = 0.0937262958253
Loss of RandomForestRegressor = 0.0971771594947
Loss of ElasticNet = 0.115479233327
Loss of Ridge = 0.0997393523542
Loss of HuberRegressor = 0.143163729309
Loss of RANSACRegressor = 0.118197640637
Loss of XGBRegressor = 0.0922075881837
Loss of GradientBoostingRegressor = 0.0948659588796


In [14]:
y_train.shape

(7979, 33)

In [15]:
def root_mean_squared_error(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true), axis=-1))

In [16]:
#Simple RNN Network Class - Using GRU Cells

class SimpleRNNNetwork(object):
    def __init__(self, input_shape, output_shape=1, layer_size=200, hidden_layers = 2):
        
        self.input_shape = input_shape
        self.output_shape = output_shape
        
        input_data = Input(shape=(input_shape,))
        hidden = Reshape((1,-1))(input_data)
        for _ in xrange(hidden_layers):
            hidden = Bidirectional(GRU(layer_size, recurrent_dropout=0.1, return_sequences=True))(hidden)
        
        hidden = Bidirectional(GRU(int(layer_size/2), recurrent_dropout=0.1))(hidden)
        
        hidden = Dropout(0.4)(hidden)
        output = Dense(output_shape, activation='linear')(hidden)
        
        self.model = Model(input_data, output)
        self.loss = root_mean_squared_error
        
    def fit(self, x, y, valid_data, batch_size=64, lr_scheme=[0.01,0.005,0.001], epochs_scheme=[3,2,2]):
        
        for lr, epochs in zip(lr_scheme, epochs_scheme):
            print("Running {} epochs with learning rate {}".format(epochs, lr))
            self.model.compile(optimizer=Adam(lr=lr), loss=self.loss)
            self.model.fit(x=x, y=y, batch_size=batch_size, epochs=epochs, validation_data=valid_data)
            
    def predict(self, x):
        batch_size=64
        return self.model.predict(x, batch_size=batch_size)
    
    

        

In [17]:
#Normalzation
X_norm_train = normalize_features(X_train, method='uniform_symmetric')
X_norm_valid = normalize_features(X_valid, method='uniform_symmetric')

In [ ]:
#Collect Regressors
winners = []
names = losses.keys()
for name in names:
    if losses[name] < 0.12:
        winners.append(name)
        

In [19]:
winners

['GradientBoostingRegressor',
 'Ridge',
 'XGBRegressor',
 'RANSACRegressor',
 'ElasticNet',
 'RandomForestRegressor',
 'ExtraTreesRegressor']

In [34]:
winners.append("XGBRegressor")

In [35]:
winners

['GradientBoostingRegressor',
 'Ridge',
 'ExtraTreesRegressor',
 'SimpleRNNNetwork',
 'XGBRegressor']

In [37]:
#Not used in this code
from sklearn.grid_search import GridSearchCV

In [25]:
GradientBoostingRegressor()

#Attempt to create GridSearch - dictionaries of parameters
gbr_params = [
    {
    "loss":"ls",
    "learning_rate":0.1,
    "n_estimators":200,
    "max_depth":4,
    "min_samples_split":3,
    "min_samples_leaf":2,
    "subsample":0.8,
        },
    {
    "loss":"huber",
    "learning_rate":0.001,
    "n_estimators":100,
    "max_depth":6,
    "min_samples_split":2,
    "min_samples_leaf":1,
    "subsample":0.6,
    },
    {
    "loss":"quantile",
    "learning_rate":0.05,
    "n_estimators":200,
    "max_depth":4,
    "min_samples_split":4,
    "min_samples_leaf":5,
    "subsample":1,
}
]

In [26]:
#GradientBoostingRegressor test

for i, params in enumerate(gbr_params):
    print("Testing set:",(i+1))
    reg = GradientBoostingRegressor(**params)
    reg = MultiOutputRegressor(reg, n_jobs=-1)
    reg.fit(X_train, y_train)
    
    predicts=reg.predict(X_valid)
    loss = RMSE_func(y_valid, predicts)
    print("\nLoss =",loss)

Testing set: 1

Loss = 0.0974939066871
Testing set: 2

Loss = 0.11771073327
Testing set: 3

Loss = 0.148266305474


In [51]:
winners

['GradientBoostingRegressor',
 'Ridge',
 'ExtraTreesRegressor',
 'SimpleRNNNetwork',
 'XGBRegressor']

In [ ]:
Ridge()

In [27]:
#Similar to above
ridge_params = [
    {
        "alpha":1.0, 
        "fit_intercept":True, 
        "normalize":False, 
        "copy_X":True, 
        "max_iter":None, 
        "tol":0.001, 
        "solver":'auto'
    },
    {
        "alpha":10.0, 
        "fit_intercept":True, 
        "normalize":True, 
        "copy_X":True, 
        "max_iter":None, 
        "tol":0.0001, 
        "solver":'cholesky'
    },
    {
        "alpha":0.1, 
        "fit_intercept":False, 
        "normalize":True, 
        "copy_X":True, 
        "max_iter":None, 
        "tol":0.01, 
        "solver":'sag'
    },
    {
        "alpha":1.0, 
        "fit_intercept":True, 
        "normalize":True, 
        "copy_X":True, 
        "max_iter":None, 
        "tol":0.005, 
        "solver":'lsqr'
    }
]

In [28]:
for i, params in enumerate(ridge_params):
    print("Testing set:",(i+1))
    reg = Ridge(**params)
    reg = MultiOutputRegressor(reg, n_jobs=-1)
    reg.fit(X_train, y_train)
    
    predicts=reg.predict(X_valid)
    loss = RMSE_func(y_valid, predicts)
    print("\nLoss =",loss)

Testing set: 1

Loss = 0.0983965932797
Testing set: 2

Loss = 0.107291292813
Testing set: 3

Loss = 0.143402379367
Testing set: 4

Loss = 0.100605561783


In [60]:
winners

['GradientBoostingRegressor',
 'Ridge',
 'ExtraTreesRegressor',
 'SimpleRNNNetwork',
 'XGBRegressor']

In [68]:
xgb.XGBRegressor()

xgb_params = [{
    "max_depth":6,
    "learning_rate":0.1,
    "n_estimators":100,
    "silent":True,
    "objective":'reg:linear',
    "nthread":-1,
    "gamma":0,
    "min_child_weight":1,
    "max_delta_step":0,
    "subsample":1,
    "colsample_bytree":1,
    "colsample_bylevel":1,
    "reg_alpha":0,
    "reg_lambda":1,
    "scale_pos_weight":1,
    "base_score":0.5
},
    {
    "max_depth":3,
    "learning_rate":0.1,
    "n_estimators":200,
    "silent":True,
    "objective":"reg:linear",
    "nthread":-1,
    "gamma":0.001,
    "min_child_weight":2,
    "max_delta_step":1,
    "subsample":0.7,
    "colsample_bytree":0.4,
    "colsample_bylevel":0.4,
    "reg_alpha":0.1,
    "reg_lambda":0.1,
    "scale_pos_weight":1,
    "base_score":0.5
},
    {
    "max_depth":4,
    "learning_rate":0.01,
    "n_estimators":150,
    "silent":True,
    "objective":'reg:linear',
    "nthread":-1,
    "gamma":0.01,
    "min_child_weight":3,
    "max_delta_step":0.5,
    "subsample":0.9,
    "colsample_bytree":0.7,
    "colsample_bylevel":0.7,
    "reg_alpha":0.3,
    "reg_lambda":0.15,
    "scale_pos_weight":1,
    "base_score":0.7
},
    {
    "max_depth":7,
    "learning_rate":0.001,
    "n_estimators":250,
    "silent":True,
    "objective":'reg:linear',
    "nthread":-1,
    "gamma":0.05,
    "min_child_weight":4,
    "max_delta_step":2,
    "subsample":1,
    "colsample_bytree":1,
    "colsample_bylevel":1,
    "reg_alpha":0.01,
    "reg_lambda":0.01,
    "scale_pos_weight":0.8,
    "base_score":0.5
},
]

In [69]:
for i, params in enumerate(xgb_params):
    print("Testing set:",(i+1))
    reg = xgb.XGBRegressor(**params)
    reg = MultiOutputRegressor(reg, n_jobs=-1)
    reg.fit(X_train, y_train)
    
    predicts=reg.predict(X_valid)
    loss = RMSE_func(y_valid, predicts)
    print("\nLoss =",loss)

Testing set: 1

Loss = 0.120865871822
Testing set: 2

Loss = 0.113375108287
Testing set: 3

Loss = 0.181657261922
Testing set: 4

Loss = 0.360203318585


In [17]:
#Class as RNN Network but using LSTM cells
class SimpleLSTMNetwork(object):
    def __init__(self, input_shape, output_shape=33, layer_size=200, hidden_layers = 2):
        
        self.input_shape = input_shape
        self.output_shape = output_shape
        
        input_data = Input(shape=(input_shape,))
        hidden = Reshape((1,-1))(input_data)
        hidden = GaussianNoise(1)(hidden)
        hidden = BatchNormalization()(hidden)
        for _ in xrange(hidden_layers):
            hidden = Bidirectional(LSTM(layer_size, recurrent_dropout=0.2, dropout=0.4, return_sequences=True))(hidden)
        
        hidden = Bidirectional(LSTM(int(layer_size/2), recurrent_dropout=0.2))(hidden)
        
        hidden = Dropout(0.4)(hidden)
        output = Dense(output_shape, activation='linear')(hidden)
        
        self.model = Model(input_data, output)
        self.loss = root_mean_squared_error
        
    def fit(self, x, y, valid_data, batch_size=64, lr_scheme=[0.01,0.005,0.001], epochs_scheme=[4,3,2]):
        
        for lr, epochs in zip(lr_scheme, epochs_scheme):
            print("Running {} epochs with learning rate {}".format(epochs, lr))
            self.model.compile(optimizer=Adam(lr=lr), loss=self.loss)
            self.model.fit(x=x, y=y, batch_size=batch_size, epochs=epochs, validation_data=valid_data)
            
    def predict(self, x):
        batch_size=64
        return self.model.predict(x, batch_size=batch_size)
    


In [21]:
slstm_nets = []
for _ in xrange(5):
    slstm_nets.append(SimpleLSTMNetwork(X_train.shape[1], y_train.shape[1], layer_size=250, hidden_layers=3))

In [20]:
batch_size = 5
cross_vals = X_train.shape[0]//batch_size

In [ ]:
# In learning time you can check results - simple cv evaluation
for i in xrange(batch_size):
    x_tre = X_norm_train[i*cross_vals:(i+1)*cross_vals]
    y_tre = y_train[i*cross_vals:(i+1)*cross_vals]
    slstm_nets[i].fit(x_tre, y_tre, (X_norm_valid, y_valid))

In [18]:
#Function for average output prediction
def average_predict(predicts):        
    preds = np.mean(predicts, axis=0)
    return preds

In [31]:
X_valid.shape

(744, 42)

In [34]:
predicts = average_predict(X_valid)

In [35]:
predicts.shape

(17447, 33)

In [36]:
loss = RMSE_func(y_valid, predicts)
print("Loss =",loss)

Loss = 0.13348235922


In [ ]:
#Check of AdaBoostRegressor 
params = {
    "max_depth":4,
    "learning_rate":0.01,
    "n_estimators":150,
    "silent":True,
    "objective":'reg:linear',
    "nthread":-1,
    "gamma":0.01,
    "min_child_weight":3,
    "max_delta_step":0.5,
    "subsample":0.9,
    "colsample_bytree":0.7,
    "colsample_bylevel":0.7,
    "reg_alpha":0.3,
    "reg_lambda":0.15,
    "scale_pos_weight":1,
    "base_score":0.7
}

reg = xgb.XGBRegressor(**params)

ada = AdaBoostRegressor(reg, n_estimators=50)
regress = MultiOutputRegressor(ada, n_jobs=-1)

regress.fit(X_train, y_train)

predicts = regress.predict(X_valid)

loss = RMSE_func(y_valid, predicts)
print("Loss =", loss)

In [18]:
model_count = 4
y_split = np.ceil(y_train.shape[1]/4.)
y_split

y_model_1 = y_train[:,0:9]
y_model_2 = y_train[:,9:18]
y_model_3 = y_train[:,18:27]
y_model_4 = y_train[:,27:]

In [19]:
y_model_1.shape, y_model_2.shape, y_model_3.shape, y_model_4.shape

((37928, 9), (37928, 9), (37928, 9), (37928, 6))

In [23]:
# Last modification - AdaBoostRegressor used on winners
gb_params =  {
    "loss":"ls",
    "learning_rate":0.01,
    "n_estimators":250,
    "max_depth":6,
    "min_samples_split":2,
    "min_samples_leaf":3,
    "subsample":0.9,
        }

model_1 = GradientBoostingRegressor(**gb_params)

rid_params = {
        "alpha":1.5, 
        "fit_intercept":True, 
        "normalize":True, 
        "copy_X":True, 
        "max_iter":None, 
        "tol":0.005, 
        "solver":'auto'
}

model_2 = Ridge(**rid_params)

parameter_dict = {
    "n_estimators":250,
    "silent":1,
    "nthread":-1,
    "gamma":0.01,
    "max_depth":7,
    "min_child_weight":2,
    "max_delta_step":0.5,
    "reg_lambda":0.2,
    "reg_alpha":0.2,
    'subsample': 0.8,
    'colsample_bylevel': 1,
    'colsample_bytree': 1,
}


model_3 = xgb.XGBRegressor(**parameter_dict)

model_4 = SimpleRNNNetwork(X_train.shape[1], y_valid.shape[1], layer_size=250, hidden_layers=3)

#model_1 = AdaBoostRegressor(model_1, n_estimators=10)
model_2 = AdaBoostRegressor(model_2, n_estimators=20)

model_1 = MultiOutputRegressor(model_1, n_jobs=-1)
model_2 = MultiOutputRegressor(model_2, n_jobs=-1)
model_3 = MultiOutputRegressor(model_3, n_jobs=-1)


In [21]:
regressors

[ExtraTreesRegressor(bootstrap=True, criterion='mse', max_depth=10,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=4,
           min_samples_split=3, min_weight_fraction_leaf=0.0,
           n_estimators=20, n_jobs=1, oob_score=False, random_state=None,
           verbose=0, warm_start=False),
 RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=10,
            max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=4,
            min_samples_split=3, min_weight_fraction_leaf=0.0,
            n_estimators=20, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False),
 ElasticNet(alpha=1.5, copy_X=True, fit_intercept=True, l1_ratio=0.1,
       max_iter=1000, normalize=True, positive=False, precompute=False,
       random_state=None, selection='cyclic', tol=0.0001, warm_start=True),
 Ridge(alpha=0.8, copy_X=True, fit_intercept=True, max_iter=None,

In [38]:
y_val_1 = y_valid[:,0:9]
y_val_2 = y_valid[:,9:18]
y_val_3 = y_valid[:,18:27]
y_val_4 = y_valid[:,27:]

In [39]:
X_train.shape

(7979, 42)

In [ ]:
models = [model_1, model_2,model_3, model_4]

count = 0

for model in models:
    print("Model nb",(count+1),"fitting")
    if count < 3:
        model.fit(X_train, y_train)
    else:
        model.fit(X_train, y_train, valid_data=(X_valid, y_valid))
    print("\tFinished")
    count += 1

In [19]:
model_5 = SimpleLSTMNetwork(X_train.shape[1], y_valid.shape[1], layer_size=200, hidden_layers=5)
model_5.fit(X_train, y_train, valid_data=(X_valid, y_valid))

Running 4 epochs with learning rate 0.01
Train on 54632 samples, validate on 743 samples
Epoch 1/4
54632/54632 [==============================] - 71s - loss: 0.0987 - val_loss: 0.0996
Epoch 2/4
54632/54632 [==============================] - 67s - loss: 0.0903 - val_loss: 0.0956
Epoch 3/4
54632/54632 [==============================] - 71s - loss: 0.0914 - val_loss: 0.0986
Epoch 4/4
54632/54632 [==============================] - 71s - loss: 0.0924 - val_loss: 0.0993
Running 3 epochs with learning rate 0.005
Train on 54632 samples, validate on 743 samples
Epoch 1/3
54632/54632 [==============================] - 75s - loss: 0.0922 - val_loss: 0.0949
Epoch 2/3
54632/54632 [==============================] - 72s - loss: 0.0908 - val_loss: 0.0938
Epoch 3/3
54632/54632 [==============================] - 72s - loss: 0.0905 - val_loss: 0.0957
Running 2 epochs with learning rate 0.001


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/kinshiryuu-burp/anaconda3/envs/nitinol/lib/python2.7/site-packages/IPython/core/ultratb.py", line 1132, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  File "/home/kinshiryuu-burp/anaconda3/envs/nitinol/lib/python2.7/site-packages/IPython/core/ultratb.py", line 313, in wrapped
    return f(*args, **kwargs)
  File "/home/kinshiryuu-burp/anaconda3/envs/nitinol/lib/python2.7/site-packages/IPython/core/ultratb.py", line 358, in _fixed_getinnerframes
    records = fix_frame_records_filenames(inspect.getinnerframes(etb, context))
  File "/home/kinshiryuu-burp/anaconda3/envs/nitinol/lib/python2.7/inspect.py", line 1048, in getinnerframes
    framelist.append((tb.tb_frame,) + getframeinfo(tb, context))
  File "/home/kinshiryuu-burp/anaconda3/envs/nitinol/lib/python2.7/inspect.py", line 1008, in getframeinfo
    filename = getsourcefile(frame) or getfile(frame)
  File "/home/kinshiryuu-burp/anaconda3/

IndexError: string index out of range

In [26]:
pred_1 = model_1.predict(X_valid)
pred_2 = model_2.predict(X_valid)
pred_4 = model_4.predict(X_valid)
pred_5 = model_5.predict(X_valid)

In [29]:
predicts = average_predict([pred_1, pred_2, pred_4, pred_5])

In [30]:
predicts.shape

(744, 33)

In [31]:
loss = RMSE_func(y_valid, predicts)
print("Loss =",loss)

Loss = 0.0947008891887


In [40]:
import os
os.mkdir("../data/models/")

In [45]:
model_4

In [32]:
import pickle

with open("../data/models/adagb_small_1.pkl", 'wb') as f:
    pickle.dump(model_1, f)
    
with open("../data/models/adaridge_small_.pkl", 'wb') as f:
    pickle.dump(model_2, f)
    
model_4.model.save("../data/models/simplernn_small.h5")
model_5.model.save("../data/models/simplelstm_small.5")


In [33]:
# "Super" Network - hidden layers connections with first layer
class SuperLSTMNetwork(object):
    def __init__(self, input_shape, output_shape=33, layer_size=200):
        
        self.input_shape = input_shape
        self.output_shape = output_shape
        
        input_data = Input(shape=(input_shape,))
        hidden = Reshape((1,-1))(input_data)
        hidden = GaussianNoise(1)(hidden)
        hidden = BatchNormalization()(hidden)
        hidden_1 = Bidirectional(LSTM(int(layer_size), recurrent_dropout=0.2))(hidden)
        hidden_1 = Reshape((1,-1))(hidden_1)
        hidden_1 = BatchNormalization()(hidden_1)
        hidden_2 = Bidirectional(GRU(int(layer_size), recurrent_dropout=0.2))(hidden_1)
        hidden_2 = Reshape((1,-1))(hidden_2)
        h_merge = merge([hidden_1, hidden_2],mode='concat')
        h_merge = Reshape((1,-1))(h_merge)
        hidden_2 = Bidirectional(GRU(int(layer_size), recurrent_dropout=0.2))(h_merge)
        merg = merge([hidden_1, h_merge],mode='concat')
        merg = Reshape((-1,))(merg)
        merged = Dense(250,kernel_initializer='he_normal', activation='tanh')(merg)
        merged = Dropout(0.4)(merged)
        output = Dense(output_shape, activation='linear')(merged)
        
        self.model = Model(input_data, output)
        self.loss = root_mean_squared_error
        
    def fit(self, x, y, valid_data, batch_size=64, lr_scheme=[0.01,0.005,0.001], epochs_scheme=[4,3,2]):
        
        for lr, epochs in zip(lr_scheme, epochs_scheme):
            print("Running {} epochs with learning rate {}".format(epochs, lr))
            self.model.compile(optimizer=Adam(lr=lr), loss=self.loss)
            self.model.fit(x=x, y=y, batch_size=batch_size, epochs=epochs, validation_data=valid_data)
            
    def predict(self, x):
        batch_size=64
        return self.model.predict(x, batch_size=batch_size)

In [34]:
super_model = SuperLSTMNetwork(X_train.shape[1], y_train.shape[1])
super_model.model.summary()

/home/kinshiryuu-burp/anaconda3/envs/nitinol/lib/python2.7/site-packages/ipykernel_launcher.py:16: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  app.launch_new_instance()
/home/kinshiryuu-burp/anaconda3/envs/nitinol/lib/python2.7/site-packages/keras/legacy/layers.py:460: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)


____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_3 (InputLayer)             (None, 42)            0                                            
____________________________________________________________________________________________________
reshape_3 (Reshape)              (None, 1, 42)         0           input_3[0][0]                    
____________________________________________________________________________________________________
gaussian_noise_2 (GaussianNoise) (None, 1, 42)         0           reshape_3[0][0]                  
____________________________________________________________________________________________________
batch_normalization_2 (BatchNorm (None, 1, 42)         168         gaussian_noise_2[0][0]           
___________________________________________________________________________________________

/home/kinshiryuu-burp/anaconda3/envs/nitinol/lib/python2.7/site-packages/ipykernel_launcher.py:19: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.


In [35]:
super_model.fit(X_train, y_train, (X_valid, y_valid))

Running 4 epochs with learning rate 0.01
Train on 7979 samples, validate on 744 samples
Epoch 1/4
7979/7979 [==============================] - 5s - loss: 0.3632 - val_loss: 0.1691
Epoch 2/4
7979/7979 [==============================] - 2s - loss: 0.1592 - val_loss: 0.1326
Epoch 3/4
7979/7979 [==============================] - 2s - loss: 0.1618 - val_loss: 0.1377
Epoch 4/4
7979/7979 [==============================] - 2s - loss: 0.1609 - val_loss: 0.1490
Running 3 epochs with learning rate 0.005
Train on 7979 samples, validate on 744 samples
Epoch 1/3
7979/7979 [==============================] - 5s - loss: 0.1293 - val_loss: 0.1116
Epoch 2/3
7979/7979 [==============================] - 2s - loss: 0.1183 - val_loss: 0.1088
Epoch 3/3
7979/7979 [==============================] - 2s - loss: 0.1193 - val_loss: 0.1070
Running 2 epochs with learning rate 0.001
Train on 7979 samples, validate on 744 samples
Epoch 1/2
7979/7979 [==============================] - 6s - loss: 0.0945 - val_loss: 0.092

In [36]:
predicts = super_model.predict(X_valid)
loss = RMSE_func(y_valid, predicts)
print("Loss =",loss)

Loss = 0.100630556242


In [37]:
test_predict = pd.read_csv("../data/hackathon/sample_power_zeros--2015-01-01_2015-07-01.csv", index_col=0)\


In [38]:
test_predict.head()

,NPWD2372,NPWD2401,NPWD2402,NPWD2451,NPWD2471,NPWD2472,NPWD2481,NPWD2482,NPWD2491,NPWD2501,...,NPWD2792,NPWD2801,NPWD2802,NPWD2821,NPWD2851,NPWD2852,NPWD2871,NPWD2872,NPWD2881,NPWD2882
ut_ms,,,,,,,,,,,,,,,,,,,,,
1420070400000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1420074000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1420077600000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1420081200000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1420084800000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [20]:
from __future__ import print_function 
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from tqdm import trange, tqdm
import pickle

%matplotlib inline
matplotlib.rcParams['figure.figsize'] = (5, 5)

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [21]:
from hackathon.utils.utils import *
from hackathon.utils.draw_utils import *
import pandas as pd

In [27]:
test_data = parse_data("../data/test_data.csv")

In [28]:
x_columns = filter(lambda x: "NPWD" not in x 
                   and 'ut_ms' not in x, test_data.columns)
x_columns

['sunmars_km',
 'earthmars_km',
 'sunmarsearthangle_deg',
 'solarconstantmars',
 'eclipseduration_min',
 'occultationduration_min',
 'sa',
 'sx',
 'sy',
 'sz',
 'dmop_count_24h_AAAA',
 'dmop_count_24h_AACF',
 'dmop_count_24h_ADMC',
 'dmop_count_24h_AHHH',
 'dmop_count_24h_AMMM',
 'dmop_count_24h_AOOO',
 'dmop_count_24h_APSF',
 'dmop_count_24h_APWF',
 'dmop_count_24h_ASEQ',
 'dmop_count_24h_ASSS',
 'dmop_count_24h_ASXX',
 'dmop_count_24h_ATMB',
 'dmop_count_24h_ATTT',
 'dmop_count_24h_AVVV',
 'dmop_count_24h_AXXX',
 'dmop_count_24h_sum',
 'flagcomms',
 'EARTH',
 'SLEW',
 'NADIR',
 'MAINTENANCE',
 'INERTIAL',
 'ACROSS_TRACK',
 'WARMUP',
 'D1PVMC',
 'RADIO_SCIENCE',
 'SPECULAR',
 'D4PNPO',
 'D3POCM',
 'D2PLND',
 'D5PPHB',
 'D7PLTS',
 'D8PLTP',
 'SPOT',
 'D9PSPO',
 'usbx',
 'usby',
 'usbz']

In [29]:
y_columns = filter(lambda x: "NPWD"  in x, test_data.columns)
y_columns

['NPWD2372',
 'NPWD2401',
 'NPWD2402',
 'NPWD2451',
 'NPWD2471',
 'NPWD2472',
 'NPWD2481',
 'NPWD2482',
 'NPWD2491',
 'NPWD2501',
 'NPWD2531',
 'NPWD2532',
 'NPWD2551',
 'NPWD2552',
 'NPWD2561',
 'NPWD2562',
 'NPWD2691',
 'NPWD2692',
 'NPWD2721',
 'NPWD2722',
 'NPWD2742',
 'NPWD2771',
 'NPWD2791',
 'NPWD2792',
 'NPWD2801',
 'NPWD2802',
 'NPWD2821',
 'NPWD2851',
 'NPWD2852',
 'NPWD2871',
 'NPWD2872',
 'NPWD2881',
 'NPWD2882']

In [30]:
X_test = test_data[x_columns]
X_test = X_test.as_matrix()

In [31]:
pred_5 = model_5.predict(X_test)

predicts = pred_5

In [32]:
X_test = test_data[x_columns]

In [33]:
submission_df = pd.DataFrame(data=predicts, index=X_test.index, columns=y_columns)
submission_df.index = to_utms(submission_df.index)

In [34]:
submission_df.head()

,NPWD2372,NPWD2401,NPWD2402,NPWD2451,NPWD2471,NPWD2472,NPWD2481,NPWD2482,NPWD2491,NPWD2501,...,NPWD2792,NPWD2801,NPWD2802,NPWD2821,NPWD2851,NPWD2852,NPWD2871,NPWD2872,NPWD2881,NPWD2882
ut_ms,,,,,,,,,,,,,,,,,,,,,
1420070400000,0.149290,-0.000430,0.175101,0.780960,0.000741,0.001296,0.002645,0.000248,0.221351,0.005444,...,0.001339,0.001403,0.165690,0.005739,0.398400,0.000133,0.000648,0.000688,0.033043,0.003651
1420074000000,0.117741,0.002175,0.175200,0.578721,0.000759,0.000808,-0.001174,0.002201,0.188289,0.005510,...,0.000973,0.001596,0.073567,0.005750,0.227190,-0.000092,0.001009,0.000776,0.025179,0.003511
1420077600000,0.162978,0.000858,0.175434,0.843790,0.001007,0.000775,0.007436,0.001627,0.241261,0.005999,...,0.001466,0.001432,0.190875,0.005365,0.452123,-0.000393,0.001003,0.000813,0.032472,0.003943
1420081200000,0.162978,0.000858,0.175434,0.843790,0.001007,0.000775,0.007436,0.001627,0.241261,0.005999,...,0.001466,0.001432,0.190875,0.005365,0.452123,-0.000393,0.001003,0.000813,0.032472,0.003943
1420084800000,0.153190,-0.000083,0.175225,0.797116,0.000811,0.001109,0.003822,0.000641,0.226430,0.005544,...,0.001396,0.001369,0.172952,0.005633,0.413435,-0.000049,0.000745,0.000714,0.033285,0.003730


In [35]:
submission_df.to_csv('../data/hackathon/nitinol_sub4.csv')